In [1]:
import networkx as nx
import widgetsnbextension
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.util import from_networkx
from py2cytoscape.cytoscapejs import viewer as cyjs

<IPython.core.display.Javascript object>

In [2]:
import sys
sys.path.insert(0, '../')

from helper_methods import *
from source_pairs import *

In [3]:
source_G = import_source_graph_from_csv('../dumps/polish_high.csv')

In [4]:
def get_induced_subgraph_nodes(G, node, hop_size=1, nodes=set()):
    nodes.add(node)
    if hop_size > 0:
        for neighbor in G[node]:
            nodes.add(neighbor)
            nodes = get_induced_subgraph_nodes(G, neighbor, hop_size-1, nodes)
    return(nodes)

def get_induced_from_subgraph(G, node, hop_size=1):
    from_subnodes = get_induced_subgraph_nodes(source_G.reverse(copy=True), node, hop_size)
    subgraph      = G.subgraph(list(from_subnodes))
    return(subgraph)

def get_induced_to_subgraph(G, node, hop_size=1):
    to_subnodes   = get_induced_subgraph_nodes(source_G, node, hop_size)
    subgraph      = G.subgraph(list(to_subnodes))
    return(subgraph)

def get_induced_subgraph(G, node, hop_size=1):
    to_subnodes   = get_induced_subgraph_nodes(source_G, node, hop_size)
    from_subnodes = get_induced_subgraph_nodes(source_G.reverse(copy=True), node, hop_size)
    subgraph      = G.subgraph(list(from_subnodes | to_subnodes))
    return(subgraph)

def rename_attrs(G, old_attr='weight', new_attr='label'):
    new_G = nx.DiGraph()
    edges = G.edges(data=True)
    print(edges)
    for edge in edges:
        from_node, to_node, attrs = edge
        attrs[new_attr]= attrs.pop(old_attr)
        new_G.add_edge(from_node, to_node, **attrs)
    return(new_G)

In [5]:
def plot(G, node, hop_size, style, layout):
    subgraph = get_induced_subgraph(G, node, hop_size)
#     subgraph = rename_attrs(subgraph)
    g_cyjs = from_networkx(subgraph)
    cyjs.render(g_cyjs, style, layout_algorithm=layout)

In [8]:
node     = 'dwu'
hop_size = 1
layouts = ['preset','circle','concentric','breadthfirst','cose','grid']
styles  = ['default','default2','Minimal','Directed','Big Labels', 'Ripple', 'Sample1', 'Solid']
plot(source_G, node, hop_size, styles[3], layouts[1])